<div style="text-align: center; font-weight: bold; font-size: 300%">Projet INF442</div>                                                        <br /> 
<div style="text-align: center; font-weight: bold; font-size: 180%"> Deep Learning </div>                                                        <br />  
<div style="text-align: center; font-size: 150%">École Polytechnique, mai 2020</div><br />  
<div style="text-align: center; font-size: 120%">Paul Calot et Jean-Charles Layoun</div>

# Apprentissage sur le jeu de données créé 

In [1]:
# importation des données 
import torch
import numpy as np
import tqdm
from torch import optim
from torch import nn
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import os
import pandas as pd

In [2]:
path_1_1 = "trainingdata_1_aller.csv"
path_1_2 = "trainingdata_1_retour.csv"
path_2_1 = "trainingdata_2_aller.csv"
path_2_2 = "trainingdata_2_retour.csv"
path_3_1 = "trainingdata_3_aller.csv"
path_3_2 = "trainingdata_3_retour.csv"
path_4_1 = "trainingdata_4_aller.csv"
path_4_2 = "trainingdata_4_retour.csv"

In [3]:
datacsv = pd.read_csv(path_1_1)
datacsv

,1.000000000000000000e+00,1.000000000000000000e+00.1,5.600000000000000000e+02,3.000000000000000000e+00,0.000000000000000000e+00,2.000000000000000000e+00
0,1.0,1.0,560.0,3.0,0.0,3.0
1,1.0,1.0,560.0,3.0,0.0,4.0
2,1.0,1.0,576.0,19.0,0.0,0.0
3,1.0,1.0,576.0,19.0,0.0,1.0
4,1.0,1.0,591.0,34.0,0.0,1.0
...,...,...,...,...,...,...
1844,46.0,1.0,799.0,89.0,0.0,0.0
1845,46.0,1.0,799.0,89.0,0.0,1.0
1846,46.0,1.0,814.0,89.0,0.0,0.0
1847,46.0,1.0,814.0,89.0,0.0,1.0


In [4]:
#data = pd.read_csv(path).to_numpy()
#np.random.shuffle(data)
data1 = pd.read_csv(path_1_1).to_numpy()
data2 = pd.read_csv(path_2_1).to_numpy()
data3 = pd.read_csv(path_3_1).to_numpy()
data4 = pd.read_csv(path_4_1).to_numpy()
data_aller = np.concatenate((data1,data2,data3,data4))

data1r = pd.read_csv(path_1_2).to_numpy()
data2r = pd.read_csv(path_2_2).to_numpy()
data3r = pd.read_csv(path_3_2).to_numpy()
data4r = pd.read_csv(path_4_2).to_numpy()
data_retour = np.concatenate((data1r,data2r,data3r,data4r))

# may need reprocessing 

In [5]:
print("Taille du dataset aller : " + str(len(data_aller)))
print("Taille du dataset aller : " + str(len(data_retour)))

Taille du dataset aller : 7398
Taille du dataset aller : 6936


In [6]:
# splitting :
data_taken = 3
def preprocessed(data):
    """
    this algorithm aims to prepare data for the deeplearning algo that follows
    that implie :
        0. Shuffling the data
        1. Splitting the data into a training set and a testing set
        2. Splitting ground truth / features
        3. Converting to torch.Tensors
        4. Normalizing data
        4. Converting into datasets
        5. Returning the datasets
    """
        # shuffling
    np.random.shuffle(data)
        
        # splitting 60 - 40
    splittinglimit = int(0.6*len(data))
    training_set = data[:splittinglimit]
    testing_set = data[splittinglimit:]
        
        # splitting ground truth / features

    x_train_ = np.zeros((len(training_set),data_taken))
    y_train_ = np.zeros(len(training_set))
    for k in range(len(training_set)):
        x_train_[k][0] = training_set[k][0] # station
        x_train_[k][1] = training_set[k][2] # estimation initiale 

        x_train_[k][2] = training_set[k][4] # si en retard ou pas
        y_train_[k] = training_set[k][5]
    
    x_test_ = np.zeros((len(testing_set),data_taken))
    y_test_ = np.zeros(len(testing_set))
    for k in range(len(testing_set)): 
        x_test_[k][0] = testing_set[k][0]
        x_test_[k][1] = testing_set[k][2]
        x_test_[k][2] = testing_set[k][4]
        y_test_[k] = testing_set[k][5]
    
        # converting to torch tensors
    x_train = torch.Tensor(x_train_)
    y_train = torch.Tensor(y_train_)
    x_test = torch.Tensor(x_test_)
    y_test = torch.Tensor(y_test_)
    
        # normalizing
    x_test = nn.functional.normalize(x_test, p=2, dim=0)
    x_train = nn.functional.normalize(x_train, p=2, dim=0)
    
        # creating datasetabs    
    training_dataset = torch.utils.data.TensorDataset(x_train,y_train)
    testing_dataset = torch.utils.data.TensorDataset(x_test,y_test)
    
    return (training_dataset, testing_dataset)


In [7]:
training_dataset_aller, testing_dataset_aller = preprocessed(data_aller)
training_dataset_retour,testing_dataset_retour = preprocessed(data_retour)


In [8]:
print("quantité de données")
print(len(training_dataset_aller))
print(len(testing_dataset_aller))
print(len(training_dataset_retour))
print(len(testing_dataset_retour))
print(training_dataset_aller[1])

quantité de données
4438
2960
4161
2775
(tensor([0.0073, 0.0213, 0.0000]), tensor(4.))


## Création d'un réseau de neurone


|                         | nombre de données entrainement | nombre de données test | 
| ------------------------|--------------------------------| -----------------------|
| aller           |        4438                |       2960          | 
| retour          |                 4161         |    2775                | 



In [9]:
# definition of a testloader
#testloader = torch.utils.data.DataLoader(test_data, len(test_data), shuffle=True)

def train_epoch(net, opt, criterion, trainloader, batch_size=50): # pour entraîner sur un epoch
    net.train()
    losses = []
    for x_batch, y_batch in trainloader:
        opt.zero_grad()
        # Forward
        y_comp = net(x_batch)
        # Compute diff
        loss = criterion(y_comp, y_batch)
        # Compute gradients
        loss.backward()
        # update weights
        opt.step()
        losses.append(loss.data.numpy())
    return losses

def accuracy(net, dataset, mins): # pour calculer la précision
    net.eval() # pass to model to evaluation mode
    correct = 0
    total = 0
    for data in dataset:
        points, labels = data
        outputs = net(points)
        #print(outputs, labels)
        total += labels.size(0)
        
        for k in range(labels.size(0)):
            if(abs(int(outputs[k].item()) - labels[k].item()) < mins+1):
                correct +=1
    net.train() # pass the model to training mode
    return correct / total


In [10]:
class Net(nn.Module):
    def __init__(self, deepness = 10, weights_nb = 256):
        super(Net, self).__init__()
        self.inputlayer = nn.Linear(data_taken,weights_nb)
        liste = []
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        for k in range(deepness):
            liste.append(nn.Linear(weights_nb,weights_nb))
            liste.append(self.relu)
        self.hiddenlayers = nn.Sequential(*liste)
        self.outputlayer = nn.Linear(weights_nb,1)
        
        
    def forward(self, x):
        # intput layer
        x = self.inputlayer(x)
        x = self.relu(x)
        # hidden layers
        x = self.hiddenlayers(x)
        # output layer
        x = self.outputlayer(x)
       #x = self.sigmoid(x)

        return x


In [11]:
net = Net() # test
net.eval()

Net(
  (inputlayer): Linear(in_features=3, out_features=256, bias=True)
  (relu): ReLU()
  (sigmoid): Sigmoid()
  (hiddenlayers): Sequential(
    (0): Linear(in_features=256, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=256, bias=True)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=256, bias=True)
    (5): ReLU()
    (6): Linear(in_features=256, out_features=256, bias=True)
    (7): ReLU()
    (8): Linear(in_features=256, out_features=256, bias=True)
    (9): ReLU()
    (10): Linear(in_features=256, out_features=256, bias=True)
    (11): ReLU()
    (12): Linear(in_features=256, out_features=256, bias=True)
    (13): ReLU()
    (14): Linear(in_features=256, out_features=256, bias=True)
    (15): ReLU()
    (16): Linear(in_features=256, out_features=256, bias=True)
    (17): ReLU()
    (18): Linear(in_features=256, out_features=256, bias=True)
    (19): ReLU()
  )
  (outputlayer): Linear(in_features=256, out_features=1, bias=Tr

In [12]:
def test_model(train_data, test_data, model, batch_size = 64, epochs = 10, lr = 1e-4):
    """
    this function returns a list of precision - 1 precision / epoch - for the given training and testing data, for the given model
    """
    testloader = torch.utils.data.DataLoader(test_data, 1, shuffle=False)
    trainloader = torch.utils.data.DataLoader(train_data, batch_size, shuffle=True)
    net = model
    
    # criterion for the last and optimizer
    def criterion(x,y):
        n = len(x)
        s = 0
        for k in range(n):
            s+=(x[k]-y[k])*(x[k]-y[k])
        return s/n
    
    opt = torch.optim.Adam(net.parameters(),lr,betas=(0.9, 0.999),eps=1e-08)
    #
    accuracy_list = []
    #accuracy_list_train = []
    for k in tqdm.tqdm(range(epochs)):
        net.train()
        train_epoch(net, opt, criterion, trainloader, batch_size = batch_size)
        prec=accuracy(net, dataset = testloader, mins = 0)
        accuracy_list.append(prec)
        #accuracy_list_train.append(accuracy(net, dataset = trainloader))
    return (net , accuracy_list) # this way we can keep the trained net and test it some more after

In [13]:
"""
training_dataset_aller
testing_dataset_aller
training_dataset_retour
testing_dataset_retour
"""
 # the NN that we'll use
net = Net(deepness = 1, weights_nb = 128)
net, accuracy_list = test_model(training_dataset_aller, testing_dataset_aller, net, batch_size = 16, epochs = 10, lr=0.0001)


100%|██████████| 10/10 [00:10<00:00,  1.09s/it]


In [14]:
net.eval()

Net(
  (inputlayer): Linear(in_features=3, out_features=128, bias=True)
  (relu): ReLU()
  (sigmoid): Sigmoid()
  (hiddenlayers): Sequential(
    (0): Linear(in_features=128, out_features=128, bias=True)
    (1): ReLU()
  )
  (outputlayer): Linear(in_features=128, out_features=1, bias=True)
)

In [15]:
print(accuracy_list)

[0.2760135135135135, 0.2760135135135135, 0.27533783783783783, 0.23141891891891891, 0.2722972972972973, 0.2760135135135135, 0.23141891891891891, 0.23141891891891891, 0.23141891891891891, 0.23141891891891891]


In [16]:
# test précision RATP :
nb_0 = 0
nb_1 = 0
nb_3 = 0
nb_10 = 0
total = 0
for k in range(len(testing_dataset_aller)):
    total +=1
    if(testing_dataset_aller[k][1]<1): # exact
        nb_0+=1
    if(testing_dataset_aller[k][1]<2): # à 3 mins
        nb_1+=1
    if(testing_dataset_aller[k][1]<4): # à 3 mins
        nb_3+=1
    if(testing_dataset_aller[k][1]<11): # à 3 mins
        nb_10+=1
print("Test sur le le testing dataset aller : ")
print("précision RATP à 0 min : " + str(nb_0/total))
print("précision RATP à 1 min : " + str(nb_1/total))
print("précision RATP à 3 min : " + str(nb_3/total))
print("précision RATP à 10 min : " + str(nb_10/total))

Test sur le le testing dataset aller : 
précision RATP à 0 min : 0.16925675675675675
précision RATP à 1 min : 0.4452702702702703
précision RATP à 3 min : 0.8057432432432432
précision RATP à 10 min : 0.985472972972973


In [17]:
# précision notre modèle à 
print("Test sur le le testing dataset aller : ")
testloader = torch.utils.data.DataLoader(testing_dataset_aller, 1, shuffle=False)
print("précision modèle à 0 min : " + str(accuracy(net, testloader, 0)))
print("précision modèle à 1 min : " + str(accuracy(net, testloader, 1)))
print("précision modèle à 3 min : " + str(accuracy(net, testloader, 3)))
print("précision modèle à 10 min : " + str(accuracy(net, testloader, 10)))

Test sur le le testing dataset aller : 
précision modèle à 0 min : 0.23141891891891891
précision modèle à 1 min : 0.6364864864864865
précision modèle à 3 min : 0.9074324324324324
précision modèle à 10 min : 0.9824324324324324


In [18]:
print("Test sur le training dataset aller : ")
testloader = torch.utils.data.DataLoader(training_dataset_aller, 1, shuffle=False)
print("précision modèle à 0 min : " + str(accuracy(net, testloader, 0)))
print("précision modèle à 1 min : " + str(accuracy(net, testloader, 1)))
print("précision modèle à 3 min : " + str(accuracy(net, testloader, 3)))
print("précision modèle à 10 min : " + str(accuracy(net, testloader, 10)))

Test sur le training dataset aller : 
précision modèle à 0 min : 0.2273546642631816
précision modèle à 1 min : 0.6297881928796756
précision modèle à 3 min : 0.9105452906714736
précision modèle à 10 min : 0.9831004957187922


On est donc meilleur pour prédire sans "gros retard" que la RATP. Avec davantage de données, on pourrait probablement encore mieux faire !

### Généralisation au sens retour ?

In [19]:
print("Test sur le testing dataset retour : ")
nb_0 = 0
nb_1 = 0
nb_3 = 0
nb_10 = 0
total = 0
for k in range(len(testing_dataset_retour)):
    total +=1
    if(testing_dataset_retour[k][1]<1): # exact
        nb_0+=1
    if(testing_dataset_retour[k][1]<2): # à 3 mins
        nb_1+=1
    if(testing_dataset_retour[k][1]<4): # à 3 mins
        nb_3+=1
    if(testing_dataset_retour[k][1]<11): # à 3 mins
        nb_10+=1
print("précision RATP à 0 min : " + str(nb_0/total))
print("précision RATP à 1 min : " + str(nb_1/total))
print("précision RATP à 3 min : " + str(nb_3/total))
print("précision RATP à 10 min : " + str(nb_10/total))

Test sur le testing dataset retour : 
précision RATP à 0 min : 0.19207207207207208
précision RATP à 1 min : 0.3981981981981982
précision RATP à 3 min : 0.7455855855855856
précision RATP à 10 min : 0.9927927927927928


In [20]:
print("Test sur le testing dataset retour : ")
testloader = torch.utils.data.DataLoader(testing_dataset_retour, 1, shuffle=False)
print("précision modèle à 0 min : " + str(accuracy(net, testloader, 0)))
print("précision modèle à 1 min : " + str(accuracy(net, testloader, 1)))
print("précision modèle à 3 min : " + str(accuracy(net, testloader, 3)))
print("précision modèle à 10 min : " + str(accuracy(net, testloader, 10)))

Test sur le testing dataset retour : 
précision modèle à 0 min : 0.21153153153153154
précision modèle à 1 min : 0.5535135135135135
précision modèle à 3 min : 0.8544144144144145
précision modèle à 10 min : 0.9978378378378379


Les résultats précédents sont à prendre avec des pincettes. Les données extraites après le processing sont partiellement biaisés (et il est difficile d'apprécier ce biais) et bruités également.

Ici le choix qui a été fait est de considérer comme période de temps pertinente la journée. C'est-à-dire que l'on s'attend à avoir une répétition de ces données d'une journée à l'autre. Ce choix est en quelque sort contraints au vu des données dont on dispose.

Dans l'idéal, on ferait des prédictions sur des périodes de temps de l'ordre de la semaine. En effet, l'échelle de l'année est plus pertinente encore, mais la quantité de données que cela représente pour pouvoir entraîner un réseau de neurones n'est pas vraisemblable. L'échelle de la semaine est "l'entre deux".

Note : les précisions peuvent changer d'une exécution à une autre. Cela est du à l'initialisation aléatoire des poids dans le réseau de neurones.